In [38]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays;

In [39]:
# T = 5
# N = 10
# seed = 1

# c = 4
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedGraph(gg)

# λ_unif = 0.5
# ρ_unif = 0.2
# λ = zeros(N,N)
# for i in CartesianIndices(λ)
#     if !iszero(g.A[i])
#         λ[i] = rand()
#     end
# end
# λ = sparse(λ)
# ρ = rand(N)
# γ = 0.8

T = 7
N = 2
seed = 6

A = [0 1; 1 0]
g = IndexedGraph(A)

λ_unif = 0.7
ρ_unif = 0.6
λ = sparse(λ_unif .* A)
# λ = sparse([0 0; λ_unif 0])
ρ = fill(ρ_unif, N)
γ = 0.5

sis = SIS_heterogeneous(λ, ρ, T; γ);
bp_obs = mpbp(sis);

In [40]:
g.A

2×2 SparseMatrixCSC{Int64, Int64} with 2 stored entries:
 ⋅  1
 1  ⋅

In [41]:
obs_times = collect(0:T)
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp_obs, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [42]:
X

2×8 Matrix{Int64}:
 2  2  2  2  2  1  2  1
 1  1  2  1  2  2  2  1

In [43]:
λinit = 0.3
ρinit = 0.3

A_complete = ones(N,N) - I
g_complete = IndexedGraph(A_complete)
λ_complete = sparse(λinit.*A_complete)
ρ_complete = fill(ρinit, N)

sis_inf = SIS_heterogeneous(g_complete, λ_complete, ρ_complete, T; γ, ϕ=deepcopy(bp_obs.ϕ))
bp_inf = mpbp(sis_inf);

In [44]:
svd_trunc = TruncBond(10)
iterate!(bp_inf, maxiter=1; svd_trunc, tol=1e-12, damp=0.5)

(1, CB_BP{ProgressMeter.ProgressUnknown, MatrixProductBP.var"#138#141"}(ProgressMeter.ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x0000000000000310) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (8, 267022608, 0)), 0, 1, false, Int64[], 1.714642784843e9, 1.714642784843e9, 1.714642784843e9)), [[[1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5], [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]], [[2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0], [1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0]]], [0.5], MatrixProductBP.var"#138#141"()))

In [45]:
bp_inf.μ[1].tensors[1]

1×3×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.00177589  0.608543  0.608546

[:, :, 2, 1] =
 -0.608543  -0.00177589  -8.39125e-6

[:, :, 1, 2] =
 -0.00177589  0.608543  -0.608546

[:, :, 2, 2] =
 -0.608543  -0.00177589  8.39125e-6

In [46]:
svd_trunc = TruncBond(10)

iters, cb = inference_parameters(bp_inf, method=2, maxiter=50, λstep=0.05, ρstep=0.05);

┌ Error: NaN in tensor train
└ @ MatrixProductBP C:\Users\fefif\Desktop\PoliTO\03 Thesis\Code\MatrixProductBP.jl\src\sis_inference.jl:23


ArgumentError: ArgumentError: invalid argument #4 to LAPACK call

In [47]:
bp_inf.w[1][1]

SIS_heterogeneousFactor{Float64}([0.3], 0.3)

In [48]:
@show cb.data

UndefVarError: UndefVarError: cb not defined

In [49]:
B3 = [0.0;;;; 0.0;;;;; 0.0;;;; 0.0;;;;;; 0.0;;;; 0.0;;;;; 0.0;;;; 0.0]

1×1×1×2×2×2 Array{Float64, 6}:
[:, :, 1, 1, 1, 1] =
 0.0

[:, :, 1, 2, 1, 1] =
 0.0

[:, :, 1, 1, 2, 1] =
 0.0

[:, :, 1, 2, 2, 1] =
 0.0

[:, :, 1, 1, 1, 2] =
 0.0

[:, :, 1, 2, 1, 2] =
 0.0

[:, :, 1, 1, 2, 2] =
 0.0

[:, :, 1, 2, 2, 2] =
 0.0